# Prueba de concepto de embedings de frases para calcular similaridad. Las recomendaciones se hacen en función de las overviews de las peliculas del dataset comparada con la overview de la pelicula que se da de input.

In [ ]:
!pip install sentence_transformers
a = "purple is the best city in the forest"
b = "there is an art to getting your way and throwing bananas on to the street is not it"
c= "it is not often you find soggy bananas on the street"
d = "green should have smelled more tranquil but somehow it just tasted rotten"
e= "joyce enjoyed eating pancakes with ketchup"
f = "as the asteroid hurtled toward earth becky was upset her dentist appointment had been canceled"
g = "to get your way you must not bombard the road with yellow fruit"

from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

sentence_embeddings = model.encode([a, b, c, d, e, f, g])

print(sentence_embeddings.shape)

from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Calculte similarities ( Will be stored in an array )
scores = np.zeros((sentence_embeddings.shape[0], sentence_embeddings.shape[0]))
for i in range(sentence_embeddings.shape[0]):
    scores[i, :] = cosine_similarity(
        [sentence_embeddings[i]],
        sentence_embeddings ) [0]

print(scores)

import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10,9))
labels = ['Doc 1','Doc 2','Doc 3','Doc 4', 'Doc 5', 'Doc 6', 'Doc 7']
sim_map = sns.heatmap(scores, xticklabels = labels, yticklabels = labels, annot = True)

# Cargo el dataset de peliculas

In [12]:
import pandas as pd
df = pd.read_csv('movies_metadata.csv')

<ipython-input-12-eb29634ceff4>:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('movies_metadata.csv')


In [13]:
# Columna de overviews
film_overviews = df['overview'].tolist()


In [14]:
def calculate_similarity(input_sentence, document_embeddings):
    input_embedding = model.encode([input_sentence])
    scores = cosine_similarity(input_embedding, document_embeddings)[0]
    return scores

In [15]:
from sentence_transformers import SentenceTransformer# esta librería la usaré para calcular los embedings de los overviews

model = SentenceTransformer('bert-base-nli-mean-tokens')

In [5]:
!pip install gradio==4.14.0

In [16]:
import gradio as gr
import random

# Assume you have already loaded your dataframe and preprocessed it
# df = ...

# Select the first 100 films for testing
df_subset = df.head(100)

# Gradio function to calculate similarity and return top 5 similar films
def find_similar_films(film_name):
    # Get the overview of the specified film
    overview = df_subset[df_subset['original_title'] == film_name]['overview'].values[0]

    # Calculate similarity scores
    film_overview_embeddings = model.encode(df_subset['overview'])
    similarity_scores = calculate_similarity(overview, film_overview_embeddings)

    # Create a dataframe with film titles and their similarity scores
    result_df = pd.DataFrame({'Film': df_subset['original_title'], 'Similarity Score': similarity_scores})

    # Sort the dataframe by similarity scores in descending order
    result_df = result_df.sort_values(by='Similarity Score', ascending=False)

    # Select the top 5 films with the highest similarity scores
    top_5_films = result_df.head(5)
    print(top_5_films)
    # Return the top 5 similar films as a formatted string
    return "\n".join(top_5_films['Film'].tolist())

# Create Gradio Interface
iface = gr.Interface(fn=find_similar_films, inputs="text", outputs="text")

# Launch the Gradio interface
iface.launch()








Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://c853b8432396cc78b2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [17]:
find_similar_films('Toy Story')

                     Film  Similarity Score
0               Toy Story          1.000000
1                 Jumanji          0.782915
94     A Midwinter's Tale          0.757052
6                 Sabrina          0.747536
71  Kicking and Screaming          0.738529


"Toy Story\nJumanji\nA Midwinter's Tale\nSabrina\nKicking and Screaming"